In [2]:
from fdh_gallica import Periodical, Search, Document
from fdh_gallica.parallel_process import iiif_urls_for_documents
from tqdm.autonotebook import tqdm
import json
import pandas as pd
from pandas.io.json import json_normalize
import itertools
from itertools import chain
from itertools import compress
import re
import wikipedia
from bs4 import BeautifulSoup
import requests
from SPARQLWrapper import SPARQLWrapper, XML

In [3]:
%matplotlib inline

get all images with relevant query

In [4]:
re_execute = False
if re_execute:
    obj = Search(all_fields='atelier Nadar', dc_type='image', dc_creator='Atelier Nadar')
    obj.execute()
    print(f'number of fetched documents: {len(obj.documents)}')
    print(f'total records {obj.total_records}')
    print('retry fetching ...')
    obj.retry()
    print(f'number of fetched documents: {len(obj.documents)}')

    df = json_normalize(json.loads(json.dumps(obj.records)))
    print('save documents to local')
    # list handling becomes a pain of we save as csv
    df.to_pickle('../data/raw_df.pkl')

In [5]:
df = pd.read_pickle('../data/raw_df.pkl')

parse identifier for easier access:

In [6]:
image_dataframe = pd.DataFrame(df['dc:identifier']\
                               .map(lambda x: x[0] if type(x) == list else x))\
                               .rename(columns={'dc:identifier':'identifier'})

df['id'] = image_dataframe

In [7]:
del df['dc:type']
del df['dc:language']
del df['dc:format']
del df['dc:identifier']
del df['dc:rights']
del df['dc:publisher']

In [8]:
#some objects are stored as list, others arent, use if else to handle
df['subject_is_list'] = df['dc:subject'].map(lambda x: type(x) == list)

In [9]:
df['len_list'] = df[df.subject_is_list]['dc:subject'].map(len)

length is non uniform ... we will have to unnest

In [10]:
subjects = list(set(list(chain.from_iterable(df['dc:subject'][df.subject_is_list])))) + df['dc:subject'][~df.subject_is_list].unique().tolist()

subjects_names = list(set(list(compress(subjects,list(map(lambda x: ('-- Portraits' in x), subjects)))))) + \
                list(set(list(compress(subjects,list(map(lambda x: ('-- Portraits' not in x) \
                                              and ('--' not in x) and (re.findall('\(\d', x) != []), subjects)))))) + \
                                    ['Nicolas II (1868-1918 ; empereur de Russie) -- Voyages']

subjects_to_exclude = list(set(list(compress(subjects,list(map(lambda x: ('-- Portraits' not in x) and ('--' in x), subjects))))) \
                           - set(['Nicolas II (1868-1918 ; empereur de Russie) -- Voyages']))

In [11]:
def title_filter(x):
    try:
        result = x.split(':')[0].strip()
        result =  re.sub('[\[\](".")]', ' ', result)
    except:
        result = x

    return result

In [12]:
def subject_filter(x):
    if type(x) == str:
        if x in subjects_to_exclude:
            result = ''
        else:
            result = [x]
    elif type(x) == list:
        temp = []
        for s in x:
            if s not in subjects_to_exclude:
                temp.append(s)
        result = temp
    else:
        result = []
        
        
    return result 

In [13]:
df2 = df[['id','dc:subject','dc:title']]
df2['subjects'] = df2['dc:subject'].apply(subject_filter)

/Users/lguillain/opt/anaconda3/envs/fdh/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
df2['title'] = df2['dc:title'].apply(title_filter)
df2['names'] = df2['subjects'].apply(lambda x: list(compress(x,list(map(lambda y: y in subjects_names, x)))))
df2 = df2[['id','dc:title','title','subjects','names']]

/Users/lguillain/opt/anaconda3/envs/fdh/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/lguillain/opt/anaconda3/envs/fdh/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
def get_name(x):
    if ('(' in x) or ('--' in x) or (',' in x):
        return x.split('--')[0].strip()
    else:
        return ''

In [16]:
named_subjects = df2[df2.names.apply(lambda x: len(x)>0)]
named_subjects['names'] = named_subjects.subjects.apply(lambda x: list(map(get_name, x)))
named_subjects.to_pickle('data/named_subjects.pkl')

/Users/lguillain/opt/anaconda3/envs/fdh/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
pre_df = named_subjects.explode('names')
pre_df = pre_df[['id','names']]
id_name = pre_df.groupby('names').apply(lambda x: x.id.tolist())
id_name =  id_name.rename('id').reset_index()
id_name = id_name.iloc[1:]
id_name  = id_name.rename(columns={'names':'name'})
id_name.to_pickle('data/id_name.pkl')

In [18]:
named_singles = df2[df2.names.apply(lambda x: len(x)==1)]

In [19]:
named_singles['name'] = named_singles.names.apply(lambda x: get_name(x[0]))
#named_individuals['name'] = named_individuals.name.apply(get_name)
named_singles = named_singles.drop(['names', 'dc:title'], axis=1)

/Users/lguillain/opt/anaconda3/envs/fdh/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
named_singles.to_pickle('data/named_singles.pkl')